### Importaciones

In [1]:
import sys
import time  # Solo para simular tiempo de procesamiento, puedes quitar esto
import os
from deepface import DeepFace
import tensorflow as tf
import numpy as np
import cv2
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
print(sys.version)
print(np.__version__)
print(tf.__version__)

3.10.15 | packaged by Anaconda, Inc. | (main, Oct  3 2024, 07:22:19) [MSC v.1929 64 bit (AMD64)]
1.26.4
2.10.1


### Funciones para extraer faces

In [2]:
def extract_pixels_from_image(image: np.ndarray, facial_area: dict) -> np.ndarray:
    """
    Extrae el área de la imagen basado en las coordenadas de 'facial_area' y devuelve
    el recorte como un array en formato float64.

    Args:
        image (np.ndarray): Imagen original cargada con OpenCV en formato uint8 BGR.
        facial_area (dict): Diccionario con las coordenadas 'x', 'y', 'w', 'h'.

    Returns:
        np.ndarray: El recorte del rostro en formato float64 y en el rango [0, 1].
    """
    x, y, w, h = facial_area['x'], facial_area['y'], facial_area['w'], facial_area['h']
    
    # Ajustar las coordenadas para evitar índices negativos
    x = max(0, x)
    y = max(0, y)
    
    # Recortar la imagen en la región del área facial
    face_crop = image[y:y+h, x:x+w]

    # Convertir el recorte a float64 y escalar a [0, 1]
    face_crop_float64 = face_crop.astype(np.float64) / 255.0
    
    return face_crop_float64

def extract_and_expand_faces(img_path: str, margin_ratio: float = 0.0) -> list:
    """
    Extrae rostros de una imagen y expande los bounding boxes según el margin_ratio.
    Args:
        img_path (str): Ruta de la imagen para procesar.
        margin_ratio (float): Proporción de expansión del bounding box. 
    Returns:
        List[Dict[str, Any]]: Lista de rostros detectados con bounding boxes ajustados.
    """
    # Cargar la imagen
    image = cv2.imread(img_path)

    # Detectar rostros con DeepFace usando el modelo RetinaFace
    faces = DeepFace.extract_faces(
        img_path=image,
        detector_backend='retinaface',
        enforce_detection=False
    )

    # Expande los bounding boxes
    for face_data in faces:
        facial_area = face_data['facial_area']
        
        # Obtener coordenadas originales
        x, y, w, h = facial_area['x'], facial_area['y'], facial_area['w'], facial_area['h']
        
        # Calcular margen adicional basado en el ratio
        margin_x = int(w * margin_ratio)
        margin_y = int(h * margin_ratio)
        
        # Expande el bounding box con el margen calculado
        facial_area['x'] = x - margin_x
        facial_area['y'] = y - margin_y
        facial_area['w'] = w + 2 * margin_x
        facial_area['h'] = h + 2 * margin_y

    for __face__ in faces:
        __face__['face'] = extract_pixels_from_image(image, __face__['facial_area'])
    
    return faces

def get_img_array_uint8(arraydata):
    img = arraydata
    #B, G, R = img.T
    #__bgr_img = np.array((B, G, R)).T
    #bgr_img = (__bgr_img*255).astype(np.uint8)
    bgr_img = (img*255).astype(np.uint8)
    return bgr_img


### Diccionario de peronas

In [3]:
def get_people_list(path_people_ref:str)->list:
    # Ruta del directorio de referencia
    ref_directory = path_people_ref

    # Lista para almacenar los diccionarios de las personas
    list_dict = []

    # Recorrer cada subdirectorio en el directorio de referencia
    for person_name in os.listdir(ref_directory):
        person_path = os.path.join(ref_directory, person_name)
    
        # Verificar si es un directorio (nombre de persona)
        if os.path.isdir(person_path):
            # Listar las imágenes dentro del subdirectorio de la persona
            images = [img for img in os.listdir(person_path) if img.endswith(".png")]
            
            # Generar rutas completas con "/" como separador
            image_paths = [os.path.join(person_path, img).replace("\\", "/") for img in images]
        
            # Crear el diccionario para esta persona
            person_dict = {
                "name": person_name,
                "images": images,
                "images_paths": image_paths
            }
        
            # Añadir a la lista
            list_dict.append(person_dict)
    
    return list_dict

# Generar la lista de personas
list_dict_people = get_people_list("./data/classroom/ref/")

# Ejemplo de acceso y salida
print(list_dict_people[2])  # Diccionario del primer elemento


{'name': 'bernal-jaime', 'images': ['img-00.png', 'img-01.png', 'img-02.png', 'img-03.png', 'img-04.png'], 'images_paths': ['./data/classroom/ref/bernal-jaime/img-00.png', './data/classroom/ref/bernal-jaime/img-01.png', './data/classroom/ref/bernal-jaime/img-02.png', './data/classroom/ref/bernal-jaime/img-03.png', './data/classroom/ref/bernal-jaime/img-04.png']}


### Iteraciones 

In [4]:
def recognize_person(
    analyze_faces: list, 
    ref_face: list, 
    metric_model_name: str = "Facenet512",
    metric_distance: str = "cosine",
    metric_threshold: float = 0.44
) -> list:
    """
    Reconoce a una persona comparando los rostros detectados en la imagen de análisis
    con el rostro de referencia, utilizando DeepFace.

    Args:
        analyze_faces (list): Lista de rostros detectados en la imagen de análisis.
        ref_face (list): Lista con el rostro objetivo (referencia).
        analyze_margin_ratio (float): Margen de expansión para los rostros analizados.
        ref_margin_ratio (float): Margen de expansión para el rostro de referencia.
        metric_model_name (str): Modelo a utilizar para la comparación.
        metric_distance (str): Métrica de distancia para la comparación.
        metric_threshold (float): Umbral para verificar si los rostros coinciden.

    Returns:
        list: Lista de diccionarios con información de los resultados para cada rostro.
    """
    if not ref_face or len(ref_face) == 0:
        print("Error: The reference face is empty.")
        return []
    
    # Obtener el rostro de referencia
    target_face_data = ref_face[0]
    target_face = target_face_data['face']
    array_bgr_target_face = (target_face * 255.0).astype(np.uint8)

    results = []
    total_faces = len(analyze_faces)

    # Iterar sobre cada rostro detectado en la imagen de análisis
    for i, face_data in enumerate(analyze_faces):
        # Progreso de la comparación
        loading = f"{(i / (total_faces - 1)) * 100:.3f}%"
        print("\r" + loading, end="")

        facial_area = face_data['facial_area']  # Bounding box
        face = face_data['face']
        array_bgr_face = (face * 255.0).astype(np.uint8)

        # Comparar el rostro detectado con el rostro de referencia
        result = DeepFace.verify(
            img1_path=array_bgr_target_face,
            img2_path=array_bgr_face,
            detector_backend="skip",
            model_name=metric_model_name,
            distance_metric=metric_distance,
            threshold=metric_threshold,
            enforce_detection=False
        )

        # Almacenar los resultados
        results.append({
            "index": i,
            "distance": result['distance'],
            "threshold": result['threshold'],
            "verified": result['verified'],
            "facial_area": facial_area
        })

    # Ordenar resultados por distancia
    results.sort(key=lambda x: x['distance'])

    # Imprimir resultados ordenados
    #for res in results:
        #print(f"Index: {res['index']} Distance: {res['distance']:.4f}")

    return results


In [5]:
def draw_verified_faces(image, result_list_dict, name_person="target"):
    """
    Dibuja los recuadros en la imagen basados en los resultados de la verificación.
    
    Args:
        image (np.ndarray): Imagen original en formato BGR.
        result_list_dict (list): Lista de resultados de verificación.
        name_person (str): Nombre del objetivo reconocido.
    
    Returns:
        np.ndarray: Imagen con los recuadros dibujados.
    """
    for result in result_list_dict:
        # Obtener coordenadas del área facial
        facial_area = result['facial_area']
        x, y, w, h = facial_area['x'], facial_area['y'], facial_area['w'], facial_area['h']

        # Verificar si el rostro es reconocido (verified=True)

        if result['verified']:
            color = (0, 255, 0)  # Verde
            confidence = 1.0 - result['distance']
            label = f"{name_person} ({confidence:.2f}%)"
            # Dibujar el recuadro y la etiqueta
            cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
            cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    return image

### Funcion para colisiones

In [6]:
def handle_face_collisions(
    analyze_faces, ref_face_list, person_name, metric_model_name="Facenet512", 
    metric_distance="cosine", initial_threshold=0.4, final_threshold=0.465, max_ref_faces=3
):
    """
    Gestiona las colisiones y coincidencias entre rostros detectados y rostros de referencia.

    Args:
        analyze_faces (list): Lista de rostros detectados en la imagen de análisis.
        ref_face_list (list): Lista de rostros de referencia (puede incluir múltiples imágenes).
        person_name (str): Nombre de la persona de referencia.
        metric_model_name (str): Modelo para la comparación.
        metric_distance (str): Métrica de distancia para la comparación.
        initial_threshold (float): Umbral inicial para la verificación.
        final_threshold (float): Umbral final para la verificación.
        max_ref_faces (int): Número máximo de rostros de referencia a comparar.

    Returns:
        list: Lista de diccionarios con los rostros verificados y su precisión.
    """
    # Limitar la cantidad de rostros de referencia
    ref_face_list = ref_face_list[:max_ref_faces]

    results = []
    
    total_ref_faces = len(ref_face_list)
    total_analyze_faces = len(analyze_faces)
    total_comparisons = total_ref_faces * total_analyze_faces

    comparison_counter = 0

    # Iterar sobre las imágenes de referencia
    for ref_face_data in ref_face_list:
        target_face = ref_face_data['face']
        array_bgr_target_face = (target_face * 255.0).astype(np.uint8)

        for face_data in analyze_faces:
            face = face_data['face']
            array_bgr_face = (face * 255.0).astype(np.uint8)

            # Primera comparación con umbral bajo
            initial_result = DeepFace.verify(
                img1_path=array_bgr_target_face,
                img2_path=array_bgr_face,
                detector_backend="skip",
                model_name=metric_model_name,
                distance_metric=metric_distance,
                threshold=initial_threshold,
                enforce_detection=False
            )
            
            if initial_result['verified']:
                # Segunda comparación con umbral más alto para confirmar
                final_result = DeepFace.verify(
                    img1_path=array_bgr_target_face,
                    img2_path=array_bgr_face,
                    detector_backend="skip",
                    model_name=metric_model_name,
                    distance_metric=metric_distance,
                    threshold=final_threshold,
                    enforce_detection=False
                )

                # Añadir el resultado a la lista si está verificado
                if final_result['verified']:
                    confidence = 1.0 - final_result['distance']
                    results.append({
                        "name": person_name,
                        "confidence": confidence,
                        "distance": final_result['distance'],
                        "facial_area": face_data['facial_area'],
                        "verified": True
                    })
            
            # Actualización del progreso
            comparison_counter += 1
            progress = (comparison_counter / total_comparisons) * 100
            sys.stdout.write(f"\rProcessing... {comparison_counter}/{total_comparisons} comparisons ({progress:.2f}%)")
            sys.stdout.flush()

    # Filtrar y resolver colisiones por precisión
    if len(results) > 1:
        results.sort(key=lambda x: x['confidence'], reverse=True)
        best_result = results[0]
        results = [best_result]

    print(f"\nFinished processing {person_name}. {len(results)} result(s) found.")
    return results


### Codigo Main

In [8]:
metrics = ["cosine", "euclidean", "euclidean_l2"]
models = ["Facenet", "Facenet512", "VGG-Face", "OpenFace", "DeepFace", "DeepID", "ArcFace", "Dlib", "SFace", "GhostFaceNet"]

# Cargar la imagen a analizar
path_analyze_image = "./data/classroom/rawdata/s2/recording_2024-11-13_10-45-09/frame-138.jpg"
image = cv2.imread(path_analyze_image)

# Extraer los rostros de la imagen a analizar
print("Extracting faces...")
faces_data = extract_and_expand_faces(path_analyze_image, 0.2)

# Obtener la lista de personas de referencia
list_dict_people = get_people_list("./data/classroom/ref-standard/")

# Iterar sobre la lista de personas para comparar
index = 0
image_with_verified_face = image.copy()

print("Comparing faces...")

for dict_person in list_dict_people:
    print(f"Person: {index + 1}/{len(list_dict_people)}")
    person_name = dict_person["name"]
    ref_image_paths = dict_person["images_paths"]

    # Establecer el número máximo de rostros de referencia
    max_ref_faces = 3  # Establece el número máximo de rostros de referencia
    limited_ref_faces = []  # Inicializamos una lista para los rostros de referencia limitados

    # Extraer rostros de todas las imágenes de referencia para esta persona
    ref_faces_list = []
    for ref_img_path in ref_image_paths:
        ref_faces_list.extend(extract_and_expand_faces(ref_img_path, 0.18))

    # Limitar el número de rostros de referencia a max_ref_faces
    limited_ref_faces = ref_faces_list[:max_ref_faces]  # Solo tomamos los primeros max_ref_faces

    # Gestionar las colisiones de rostros
    print(f"Handling collisions for {person_name}...")

    # Llamamos a la función con los rostros limitados
    result_list = handle_face_collisions(
        faces_data,  # Rostros detectados en la imagen a analizar
        limited_ref_faces,  # Pasamos solo los primeros `max_ref_faces` rostros de referencia
        person_name=person_name,
        metric_model_name="Facenet512",
        metric_distance="cosine"
    )

    # Añadir recuadros después de procesar todas las verificaciones
    print("Drawing boxes for verified faces..."f"\n{'.'*50}")

    image_with_verified_face = draw_verified_faces(image_with_verified_face, result_list, name_person=person_name)
    index += 1

# Convertir BGR a RGB para mostrar con Matplotlib
image_rgb = cv2.cvtColor(image_with_verified_face, cv2.COLOR_BGR2RGB)

# Mostrar la imagen
plt.figure(figsize=(15, 15))
plt.imshow(image_rgb)
plt.axis('on')
plt.show()


Extracting faces...
Comparing faces...
Person: 1/20
Handling collisions for aparicio-azner...
Processing... 24/24 comparisons (100.00%)
Finished processing aparicio-azner. 1 result(s) found.
Drawing boxes for verified faces...
..................................................
Person: 2/20
Handling collisions for ayala-javier...
Processing... 8/8 comparisons (100.00%)
Finished processing ayala-javier. 0 result(s) found.
Drawing boxes for verified faces...
..................................................
Person: 3/20


KeyboardInterrupt: 